### UMAP for DQM
* Load a pre-trained UMAP
* Visualize the pre-trained UMAP
* Use the pre-trained UMAP to run inferences on a new waveform
* Visualize the results on a Bokeh plot

A. Higuera (Rice University)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import umap.umap_ as umap
import pickle
import pandas as pd

In [ ]:
filename = '../model_v0_NP04_run005834_0038_dl7.pkl'

In [ ]:
loaded_reducer = pickle.load(open(filename, 'rb'))

In [ ]:
# load training data
embedding = np.load('../UMAP_embedding_model_v0_NP04_run005834_0038_dl7.npy')

In [ ]:
plt.scatter(embedding[:, 0], embedding[:, 1], cmap='Spectral', s=1)
plt.gca().set_aspect('equal', 'datalim')
plt.xlabel("UMAP latent variable 1")
plt.ylabel("UMAP latent variable 1")
plt.title('UMAP projection on taining data', fontsize=12);

In [ ]:
# load some data to do inferences 
waveforms = np.load('../waveform_data.npy')

In [ ]:
# this contains two waveforms
waveforms.shape

In [ ]:
new_embedding = loaded_reducer.transform(waveforms[0])

In [ ]:
new_embedding.shape

In [ ]:
wf_test = pd.DataFrame(new_embedding, columns=('x', 'y'))

In [ ]:
wf_test['index'] = indices = [i for i in range(len(waveforms[0]))]

In [ ]:
from io import BytesIO
from PIL import Image
import base64
from bokeh.plotting import figure, show, output_notebook, output_file, save
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper
from bokeh.palettes import Spectral10

In [ ]:
def embeddable_image(wf):
    plt.figure(figsize=(2, 0.4))  # Specify the desired figure size
    plt.plot(wf)
    plt.axis('off')  # Turn off axes for cleanliness

    buffer = BytesIO()
    plt.savefig(buffer, format='png', bbox_inches='tight', pad_inches=0)  # Save the plot to a buffer
    plt.close()  # Close the plot to free up memory
    buffer.seek(0)
    image_png = buffer.getvalue()
    image_base64 = base64.b64encode(image_png)
    return 'data:image/png;base64,' + image_base64.decode()

In [ ]:
wf_test['image'] = list(map(embeddable_image, waveforms[0]))

In [ ]:
plot_figure = figure(
    title='UMAP projection of waveforms',
    width=1000,
    height=800,
    tools=('pan, wheel_zoom, reset')
)

### Hover plots 
# Define the hover tool to use 'image' from the data source
hover = HoverTool(tooltips="""
    <div>
        <div><img src="@image" style="float: left; width:240px; margin: 5px 5px 5px 5px"/></div>
        <div><span style="font-size: 16px;">channel:</span> <span style="font-size: 18px;">@index</span></div>
    </div>
""")

plot_figure.add_tools(hover)

plot_figure.circle(
    'x',
    'y',
    source=wf_test,
    line_alpha=0.6,
    fill_alpha=1,
    size=10,
    color='blue'
)

plot_figure.xaxis.axis_label = "UMAP latent dimension 1"  # Replace "X-Axis Label" with your actual x-axis label
plot_figure.yaxis.axis_label = "UMAP latent dimension 2" 
output_file("test.html")
save(plot_figure)